In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from collections import deque
import random

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = models.Sequential()
        model.add(layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=tf.keras.optimizers.Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns the action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Assuming your environment is defined as before, with modifications to handle continuous states
environment = PrisonersDilemmaEnvironment()
state_size = 2  # Define according to your environment's state representation
action_size = 2  # 'cooperate' or 'betray'

agent1 = DQNAgent(state_size, action_size)
agent2 = DQNAgent(state_size, action_size)

def train_dqn(environment, agent1, agent2, episodes, batch_size=32):
    for e in range(episodes):
        state = environment.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):  # Maximum time steps per episode
            # Agent 1 acts
            action1 = agent1.act(state)
            # Agent 2 acts
            action2 = agent2.act(state)
            # Environment steps
            next_state, reward, done, _ = environment.step(action1, action2)
            reward1, reward2 = reward  # Unpack the rewards for each agent
            
            next_state = np.reshape(next_state, [1, state_size])
            
            # Remember and replay
            agent1.remember(state, action1, reward1, next_state, done)
            agent2.remember(state, action2, reward2, next_state, done)
            
            state = next_state
            
            if len(agent1.memory) > batch_size:
                agent1.replay(batch_size)
            if len(agent2.memory) > batch_size:
                agent2.replay(batch_size)
            
            if done:
                break
                
        print(f"Episode {e+1}/{episodes}, Score: {time}, Epsilon: {agent1.epsilon:.2}")

# Train the agents
train_dqn(environment, agent1, agent2, 1000)